In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
import json
import numpy as np
print(tf.__version__)

2.0.0


In [2]:
import sys
print(sys.executable)

/Users/knowledge/Developer/Chatbot/.venv/bin/python


## Hyperparameters

In [26]:
oov_tok = '<OOV>'
num_epochs = 30

## Load Intents

In [4]:
with open('../lambda/intents.json', 'r') as f:
    intents = json.load(f)

## Gather and Tokenize Intents, Labels to One-Hot presentation

In [5]:
intent_list = []
label_list = []
for index, intent in enumerate(intents):
    intent_list += intent['patterns']
    num_patterns = len(intent['patterns'])
    label_list += [index] * num_patterns

tokenizer = Tokenizer(oov_token = oov_tok)
tokenizer.fit_on_texts(intent_list)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(intent_list)
max_length = len(max(sequences, key=len))
vocab_size = len(word_index)
padded = pad_sequences(sequences)
padded_length = len(padded[0])
labels = tf.keras.utils.to_categorical(label_list)
num_categories = len(labels[0])

## Model

In [21]:
model = tf.keras.Sequential([
    layers.Embedding(vocab_size+1, 16, input_length=padded_length),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_categories, activation='softmax')
])
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['acc'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 8, 16)             400       
_________________________________________________________________
flatten_5 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               16512     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 2)                

In [27]:
history = model.fit(padded, labels, epochs=num_epochs, verbose=2)

Train on 10 samples
Epoch 1/30
10/10 - 0s - loss: 0.0013 - acc: 1.0000


Epoch 2/30
10/10 - 0s - loss: 0.0034 - acc: 1.0000


Epoch 3/30
10/10 - 0s - loss: 0.0022 - acc: 1.0000


Epoch 4/30
10/10 - 0s - loss: 0.0160 - acc: 1.0000


Epoch 5/30
10/10 - 0s - loss: 0.0021 - acc: 1.0000


...

Epoch 26/30
10/10 - 0s - loss: 0.0036 - acc: 1.0000


Epoch 27/30
10/10 - 0s - loss: 0.0012 - acc: 1.0000


Epoch 28/30
10/10 - 0s - loss: 0.0010 - acc: 1.0000


Epoch 29/30
10/10 - 0s - loss: 0.0074 - acc: 1.0000


Epoch 30/30
10/10 - 0s - loss: 0.0096 - acc: 1.0000


In [30]:
sentence = 'i need a loan'
sequence = tokenizer.texts_to_sequences([sentence])
padded_sequence = pad_sequences(sequence, maxlen=padded_length)
prediction = model.predict(padded_sequence)[0]
intent = intents[np.argmax(prediction)]
tag = intent['tag']
print(sentence, tag, prediction)

i need a loan loan [4.2788292e-06 9.9999571e-01]


## Export model

In [9]:
# model_json = model.to_json()
# with open('../lambda/model.json', 'w') as json_file:
#     json_file.write(model_json)
# model.save_weights('../lambda/model.h5')
# json_file.close()

# model.save('../lambda/saved_model')


# with open('../lambda/wordIndex.json', 'w') as json_file:
#     json.dump(word_index, json_file)
# json_file.close()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Assets written to: ../lambda/saved_model/assets
